# Organização da Pipeline de pré-processamento

A definição dos tratamentos e transformações dos dados foi realizada com base nos insights extraídos durante a análise exploratória (EDA). Para otimizar essa etapa, foi utilizada uma [planilha de organização](https://docs.google.com/spreadsheets/d/12IVVANl1VLA55f7QB15Y_hmrT_lGEXov8IsCHVvneR0/edit?usp=sharing), que ajudou a planejar os tratamentos e transformações necessários em cada variável.

Com o objetivo de melhorar a reprodutibilidade e escalabilidade do processo, foram desenvolvidas duas pipelines separadas: Pipeline de Tratamento e Pipeline de Transformação. Essas pipelines foram aplicadas separadamente no conjunto de treino e de teste para evitar o risco de data leakage.

### Pipeline de tratamento:

Realizada para sanar problemas de ausência de engenharia de dados. Nessa pipeline, foi buscado resolver os seguintes problemas:

* Remoção de colunas desconsideradas para o modelo
* Remover underscore e espaços vazios
* Transformar dados nos tipos corretos
* Tratar outliers
* Feature engineering: converter a variável categórica `Credit_History_Age` (tempo total de histórico de crédito do cliente) no total de meses.

### Pipeline de transformação:

Realizada para preparar os dados para os diferentes modelos de machine learning.

* Dados numéricos: substituir dados ausentes pela média
* Dados categóricos: substituir ausentes por "missing"
* Dados categóricos (encoder): passar um encoder (ex: Target Encoder)
* Todas as variáveis (scale): passar um scaler (ex: MinMax ou Robust Scaler)

# Bibliotecas

In [1]:
# Data manipulation

import pandas as pd
import numpy as np

# Data viz

import matplotlib.pyplot as plt
import seaborn as sns

# ML

from sklearn.model_selection import train_test_split

# 1. Importação dos dados

In [2]:
df = pd.read_csv('data/train.csv', low_memory=False)

**OBS (1):** nos dados de teste há uma variável a menos, que é a 'Credit_Score', inicialmente definida como o target.  
**OBS (2):** o alvo da base de dados era a variável Credit_Score, que consiste na pontuação de crédito do cliente. Nessa classificação, haviam indivíduos classificados como: Poor, Standard e Good. Para deixar o problema ainda mais próximo do proposto na concessão de crédito, optou-se por modificar as classes, como indicado abaixo.  

* Clientes classificados como `Poor` ➡ `Classe 1` (maus pagadores)  
* Clientes classifiados como `Standard` ➡ `Classe 0` (bons pagadores)  
* Clientes classifiados como `Good` ➡ `Classe 0` (bons pagadores)

**OBS (3):** o objetivo original da base de dados era prever a variável Credit_Score, que reflete a pontuação de crédito do cliente, com base em variáveis como comportamento de pagamento e dívida pendente. No entanto, o foco foi ajustado para classificar futuros clientes em bons ou maus pagadores, visando prevenir inadimplências durante o processo de concessão de crédito. As variáveis relacionadas ao comportamento dos clientes, que já constavam na base de dados, foram consideradas como informações complementares, hipoteticamente fornecidas por outro birô de crédito (ex: Serasa, SPC, etc).

In [3]:
# Modificando as classes para bons e maus pagadores

df['target'] = df['Credit_Score'].apply(lambda x: 1 if x == 'Poor' else 0)
df = df.drop('Credit_Score', axis=1)

In [4]:
# Base de dados adaptada

df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,target
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,0


# 2. Split dos dados

In [5]:
X = df.drop('target', axis=1)
y = df['target']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [7]:
# Dimensionalidade dos dados de treino

X_train.shape

(80000, 27)

In [8]:
# Dimensionalidade dos dados de teste

X_test.shape

(20000, 27)

# 2. Pipeline de tratamento

In [9]:
# Remoção de colunas desnecessárias

def drop_variaveis(df):
    lista_variaveis = ['ID', 'Customer_ID', 'SSN', 'Name', 'Delay_from_due_date', 'Num_of_Delayed_Payment']
    return df.drop(columns=lista_variaveis, axis=1)

X_train = drop_variaveis(X_train)
X_test = drop_variaveis(X_test)

print(f'Total de colunas: {X_test.shape[1]}')

Total de colunas: 21


In [10]:
# Tratamento de NaNs e conversão de dados

col_treat_num = ['Age','Num_Bank_Accounts','Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan','Num_Credit_Inquiries','Annual_Income',
                 'Monthly_Inhand_Salary','Changed_Credit_Limit', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month',
                 'Amount_invested_monthly', 'Monthly_Balance']


def tratar_dados(df, colunas):
    # Passa todas as features para object; todas as que possuem caractere != número são substituídas por NaN; depois passa todas para float
    
    df[colunas] = df[colunas].astype('object')
    
    for col in colunas:
        # Converte os valores para string para aplicar a substituição de caracteres
        df[col] = df[col].astype(str).replace(r'[^0-9.-]', '', regex=True)
        # Converte a coluna para float, transformando valores inválidos em NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df


X_train = tratar_dados(X_train, col_treat_num)
X_test = tratar_dados(X_test, col_treat_num)

#### Outliers

* Para cada variável numérica do dataset, foram calculados os percentis 1 e 99;
* Valores de uma observação maiores que o percentil 99 da variável, foram substituídos pelo valor do percentil 99;
* Valores de uma observação menores que o percentil 1 da variável, foram substituídos pelo valor do percentil 1
  
Esse processo limitou a influência de valores extremos sem removê-los, preservando a integridade do dataset.

In [11]:
# Demonstração dos outliers

for col in X_train.columns:
    # Verificando se a coluna é numérica
    if X_train[col].dtype in ['int64', 'float64']:
        # Exibindo o valor mínimo e máximo para cada coluna numérica
        print(f'Coluna: {col}')
        print(f'Mínimo: {X_train[col].min()}')
        print(f'Máximo: {X_train[col].max()}')
        print('-' * 30)

Coluna: Age
Mínimo: -500
Máximo: 8698
------------------------------
Coluna: Annual_Income
Mínimo: 7005.93
Máximo: 24177153.0
------------------------------
Coluna: Monthly_Inhand_Salary
Mínimo: 303.6454166666666
Máximo: 15204.633333333331
------------------------------
Coluna: Num_Bank_Accounts
Mínimo: -1
Máximo: 1798
------------------------------
Coluna: Num_Credit_Card
Mínimo: 0
Máximo: 1499
------------------------------
Coluna: Interest_Rate
Mínimo: 1
Máximo: 5797
------------------------------
Coluna: Num_of_Loan
Mínimo: -100
Máximo: 1496
------------------------------
Coluna: Changed_Credit_Limit
Mínimo: -6.49
Máximo: 36.97
------------------------------
Coluna: Num_Credit_Inquiries
Mínimo: 0.0
Máximo: 2594.0
------------------------------
Coluna: Outstanding_Debt
Mínimo: 0.23
Máximo: 4998.07
------------------------------
Coluna: Credit_Utilization_Ratio
Mínimo: 20.0
Máximo: 50.00000000000001
------------------------------
Coluna: Total_EMI_per_month
Mínimo: 0.0
Máximo: 82331.

In [12]:
def treat_outliers(df):
    # Selecionando todas as colunas numéricas
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    for col in numeric_cols:
        # Calculando os percentis 1 e 99
        p1 = np.percentile(df[col], q=1)
        p99 = np.percentile(df[col], q=99)
        
        # Garantir que os percentis sejam do mesmo tipo da coluna
        dtype_col = df[col].dtype
        
        # Substituindo os valores menores que o percentil 1 pelo próprio percentil 1
        df.loc[df[col] < p1, col] = dtype_col.type(p1)
        # Substituindo os valores maiores que o percentil 99 pelo próprio percentil 99
        df.loc[df[col] > p99, col] = dtype_col.type(p99)
    
    return df

In [13]:
# Aplicando a função nos dados de treino e de teste

X_train = treat_outliers(X_train)
X_test = treat_outliers(X_train)

In [14]:
# Checagem

for col in X_train.columns:
    # Verificando se a coluna é numérica
    if X_train[col].dtype in ['int64', 'float64']:
        # Exibindo o valor mínimo e máximo para cada coluna numérica
        print(f'Coluna: {col}')
        print(f'Mínimo: {X_train[col].min()}')
        print(f'Máximo: {X_train[col].max()}')
        print('-' * 30)

Coluna: Age
Mínimo: 14
Máximo: 4135
------------------------------
Coluna: Annual_Income
Mínimo: 7531.855
Máximo: 179994.47017199246
------------------------------
Coluna: Monthly_Inhand_Salary
Mínimo: 303.6454166666666
Máximo: 15204.633333333331
------------------------------
Coluna: Num_Bank_Accounts
Mínimo: 0
Máximo: 442
------------------------------
Coluna: Num_Credit_Card
Mínimo: 1
Máximo: 847
------------------------------
Coluna: Interest_Rate
Mínimo: 1
Máximo: 2874
------------------------------
Coluna: Num_of_Loan
Mínimo: -100
Máximo: 9
------------------------------
Coluna: Changed_Credit_Limit
Mínimo: -6.49
Máximo: 36.97
------------------------------
Coluna: Num_Credit_Inquiries
Mínimo: 0.0
Máximo: 2594.0
------------------------------
Coluna: Outstanding_Debt
Mínimo: 25.78
Máximo: 4809.550168
------------------------------
Coluna: Credit_Utilization_Ratio
Mínimo: 22.74693297281844
Máximo: 42.460768733718616
------------------------------
Coluna: Total_EMI_per_month
Mínimo

In [15]:
# Tratamento extra para a variável de idade

X_train.loc[X_train['Age'] > 100, 'Age'] = np.nan
X_test.loc[X_test['Age'] > 100, 'Age'] = np.nan

In [16]:
print(f'Idade mínima: {X_train['Age'].min()}')
print(f'Idade mínima: {X_train['Age'].max()}')

Idade mínima: 14.0
Idade mínima: 99.0


# 2. Pipeline de transformação

In [17]:
# Scalers e encoders